In [ ]:
!pip3 install pandas

In [3]:
!/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip3 install torch

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-2.2.2-cp39-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached MarkupSafe-2.1.5-cp39-cp39-macosx_10_9_x86_64.whl.metadata (3.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/150.8 MB 55.3 kB/s eta 0:43:01
ERROR: Exception:
Traceback (most recent

In [ ]:
!pip3 install scikit-learn

In [9]:
!pip3 install catboost

In [ ]:
!pip3 install matplotlib

# Imports

In [2]:
import pandas as pd
import catboost
import numpy as np
import matplotlib.pyplot as plt

<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [44]:
from catboost import CatBoostClassifier

In [14]:
from sklearn.preprocessing import LabelEncoder

In [42]:
from sklearn.model_selection import train_test_split

In [53]:
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, consensus_score

# Load and preprocessing data

## Load data

In [34]:
# загрузка датасетов
dataset = pd.read_csv('data/dataset.csv')
symptom_data = pd.read_csv('data/symptom_Description.csv')
symptom_precaution = pd.read_csv('data/symptom_precaution.csv')
symptom_severity = pd.read_csv('data/Symptom-severity.csv')

In [35]:
# заполнение отсутстующих значений
dataset = dataset.fillna('0')

In [36]:
for col in dataset.columns[1:]:
    dataset[col] = dataset[col].apply(lambda x: x.replace(' ', ''))

## LE

In [37]:
# Кодирование болезней и симптомов
symptom_le = LabelEncoder()
symptom_le.fit([*symptom_severity.Symptom.values, '0'])
disease_le = LabelEncoder()
disease_le.fit(dataset.Disease.unique())

LabelEncoder()

## Encode dataset 

In [38]:
# кодирование данных
for col in dataset.columns[1:]:
    dataset[col] = symptom_le.transform(dataset[col].values)
dataset[dataset.columns[0]] = disease_le.transform(dataset[dataset.columns[0]].values)

In [40]:
# переименование столбцов
dataset = dataset.rename(columns={'Disease' : 'target'})

## Division

In [41]:
# разбиение датасета на Х и у
X = dataset[dataset.columns[1:]]
y = dataset['target'].to_frame()

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Model create and train

In [ ]:
import torch.nn as nn
import torch

In [45]:
# create class nn model
class DiseaseNN(nn.Module):
    def __init__(self, num_classes : int, input_shape: int=17):
        super(self.__class__, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_shape, 300),
            nn.ReLU(),
            nn.BatchNorm1d(300),
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.BatchNorm1d(100),
            nn.Linear(100, num_classes),
            nn.
        )
    
    def forward(self, input_data):
        out = self.model(input_data)
        return out

In [46]:
# make model 
n_epochs = 30
device = 'cpu'

model = DiseaseNN().to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_func = nn.CrossEntropyLoss()

In [ ]:
# train
accuracy = metric.Accuracy('multiclass', num_classes = len(train_loader.dataset.classes))

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

for epoch in range(n_epochs):
    print("Number of epochs: ", epoch+1, "of 30")
    epoch_train_loss = []
    epoch_test_loss = []
    epoch_test_accuracy = []

    model.train(True)
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        y_pred = model(X_batch)
        loss = loss_func(y_pred, y_batch)
        loss.backward()
        opt.step()
        opt.zero_grad()
        epoch_train_loss.append(loss.item())
    train_loss.append(epoch_train_loss)

    model.train(False)
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            y_pred = model(X_batch)
            loss = loss_func(y_pred, y_batch)
            epoch_test_accuracy.append(accuracy(y_pred, y_batch))
            epoch_test_loss.append(loss)
    test_loss.append(epoch_test_loss)
    test_accuracy.append(epoch_test_accuracy)

# Check work model

In [48]:
y_pred = model.predict(X_test).reshape(-1)

In [55]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [56]:
print(f'accuracy {accuracy}')
print(f'Classification Report:\n {report}')

accuracy 0.997289972899729
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        32
           1       1.00      1.00      1.00        39
           2       0.91      1.00      0.95        41
           3       1.00      1.00      1.00        36
           4       1.00      1.00      1.00        35
           5       1.00      1.00      1.00        36
           6       1.00      1.00      1.00        44
           7       1.00      1.00      1.00        32
           8       1.00      1.00      1.00        35
           9       1.00      1.00      1.00        30
          10       1.00      1.00      1.00        31
          11       1.00      1.00      1.00        40
          12       1.00      1.00      1.00        33
          13       1.00      0.91      0.95        45
          14       1.00      1.00      1.00        35
          15       1.00      1.00      1.00        28
          16       1.00      1

# Save model

In [57]:
model.save_model(
    'model.json',
    format='json'
)